# Titanic - Machine Learning from Disaster

## Imports

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OrdinalEncoder

## Global config

In [2]:
train_csv = "encoded_data.csv"

## Load data

In [3]:
train_data = pd.read_csv(train_csv, index_col='PassengerId')

### Data dictionary

#### Categorical features

- Survived - Survival - 0 = No, 1 = Yes - **Used for evaluating whether a person survived**
- Sex - Sex 	
- Embarked - Port of Embarkation - C = Cherbourg, Q = Queenstown, S = Southampton

#### Ordinal

- Pclass - Ticket class - 1 = 1st, 2 = 2nd, 3 = 3rd

#### Numerical features

##### Discrete

- SibSp - # of siblings / spouses aboard the Titanic 	
- Parch - # of parents / children aboard the Titanic 

##### Continuous

- Age - Age in years 		
- Fare - Passenger fare 

#### Mixed/Error-prone/Unused

- PassengerId - Id of person
- Name - Name of Passenger
- Ticket - Ticket number 
- Cabin - Cabin number

In [4]:
y_column = 'Survived'

In [5]:
import os
import random
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Set warnings to not be displayed
pd.options.mode.chained_assignment = None

class Divider:
    '''
    Table divider class
    '''

    def __init__(self, input_table, important_column):
        '''
        Constructor for divider

        input_table - table to apply division on
        important_column - y_column used for predictions
        '''
        self.result = []
        self.input_table = input_table.copy()
        self.important_column = important_column
        self.connections = []
        self.index = 0
    
    def get_result(self):
        '''
        Returns the result of calculations
        '''
        return self.result

    def random_same_pk_fk(self, input_table, level, onehot=False):
        '''
        Recursively cluster randomly the columns
        level - level of recursion
        '''

        input_table = input_table.copy()
        
        # Return if no columns
        if len(input_table.columns) <= 1:
            return
        
        # Set up base table variables
        base_index = self.index
        self.index += 1
        base_index_cols = input_table.index.names
        
        # Apply clustering for every cluster of columns
        while len(input_table.columns) > 0:

            # Randomly shuffle
            self.index += 1
            mylist = np.array(range(0, len(input_table)))
            random.shuffle(mylist)

            # Randomly pick n_splits columns
            picked_columns = []
            if len(input_table.columns) > 1:
                picked_columns = input_table.sample(n=np.random.randint(1, len(input_table.columns)), axis='columns').columns
            else:
                picked_columns = input_table.columns       
            
            # Set new column names
            PK_name = 'PK' + str(level + 1) + str(self.index)
            FK_name = 'FK' + str(level + 1) + str(self.index)
            
            # Add new PK
            recursed_table = input_table[picked_columns]
            recursed_table.loc[:, PK_name] = mylist
            recursed_table.set_index(PK_name, inplace = True)
            
            # Add new FK and remove columns associated with it
            input_table.loc[:, FK_name] = mylist
            input_table = input_table.groupby(input_table.index.names + [FK_name]).mean()
            input_table = input_table.drop(picked_columns, axis = 1)
            
            # Add the connection to a list
            self.connections.append(('table' + str(level) + str(base_index), FK_name,  
                                     'table' + str(level + 1) + str(self.index), PK_name))
            
            # Append new FK table to result list
            if len(recursed_table.columns) == 1:
                # Check if you need to apply oneHotEncoding
                if onehot is True and len(recursed_table[recursed_table.columns[0]].unique()) < 7:
                    oneHotEncoder = OneHotEncoder()
                    encoded_col = pd.DataFrame(oneHotEncoder.fit_transform(recursed_table[[recursed_table.columns[0]]]).toarray())

                    # Concatenate the 2 tables
                    recursed_table = pd.concat([recursed_table.reset_index(), encoded_col], axis=1, copy=False, join='inner')

                    # Readd the index column
                    recursed_table.loc[:, PK_name] = mylist
                    recursed_table.set_index(PK_name, inplace = True)
                
                # Append single-column table to result
                self.result.append((level + 1, self.index, recursed_table))
                continue
            
            # Apply clustering recursively on smaller table
            self.random_same_pk_fk(recursed_table, level + 1, onehot=onehot)
        
        # Reset the index and set FK columns as normal columns
        input_table = input_table.reset_index()
        input_table.set_index(base_index_cols, inplace = True)
        self.result.append((level, base_index, input_table))
    
    def correlation(self, input_table, important_column, level):
        '''
        Recursively cluster most correlated columns to an "important_column"
        important_column - colum of interest, most likely to be Y
        input_table - table to apply the clustering on
        '''

        input_table = input_table.copy()

        n_splits = 3
        
        # Return if no columns
        if len(input_table.columns) == 0:
            return
        
        # Set up base table variables
        base_index = self.index
        self.index += 1
        base_index_cols = input_table.index.names

        # Calculate correlation between columns and most important column
        corr = abs(input_table.corr(method='spearman'))
        corr = corr.drop([important_column], axis = 1)
        
        # Calculate quantiles based on correlation and n_splits
        quantiles = []
        for i in range(n_splits):
            quantile = 1 - (i+1) / n_splits
            quantiles.append(corr.loc[[important_column]].T.quantile(quantile)[0])
        
        # Apply clustering for every cluster of columns
        for threshold in quantiles:
            # Randomly shuffle
            self.index += 1
            mylist = np.array(range(0, len(input_table)))
            random.shuffle(mylist)

            # Break if no columns
            if len(corr.columns) == 0 or len(input_table.columns) == 0:
                break
        
            # Pick the new important column
            new_important = corr.loc[[important_column]].idxmax(axis=1)[0]
            # Pick all columns with correlation above quantile threshold
            corr_columns = [col for col in corr.loc[[important_column]].columns if corr.loc[[important_column]][col][0] >= threshold]
           
            if len(corr_columns) == 0:
                continue

             # Set new column names
            PK_name = 'PK' + str(level + 1) + str(self.index)
            FK_name = 'FK' + str(level + 1) + str(self.index)
            
            # Add new PK
            recursed_table = input_table[corr_columns]
            recursed_table.loc[:, PK_name] = mylist
            recursed_table.set_index(PK_name, inplace = True)
            
            # Add new FK and remove columns associated with it
            input_table.loc[:, FK_name] = mylist
            input_table = input_table.groupby(input_table.index.names + [FK_name]).mean()
            input_table = input_table.drop(corr_columns, axis = 1)
            corr = corr.drop(corr_columns, axis = 1)
            
            # Add the connection to a list
            self.connections.append(('table' + str(level) + str(base_index), FK_name,  
                                     'table' + str(level + 1) + str(self.index), PK_name))
            
            # Apply clustering recursively
            self.correlation(recursed_table, new_important, level + 1)

        # Reset the index and set FK columns as normal columns
        input_table = input_table.reset_index()
        input_table.set_index(base_index_cols, inplace = True)
        self.result.append((level, base_index, input_table))

    
    def divide(self, strategy, path, onehot=False):
        '''
        Function used to divide the table
        strategy - strategy of division
        path - path to save output
        '''

        # Create output folder
        os.makedirs(path, exist_ok=True) 
        
        # Initialise fresh result and connections lists
        self.__init__(self.input_table, self.important_column)
        
        # Pick strategy
        if strategy == 'random':
            input_table = self.input_table.groupby(self.input_table.index.names + [self.important_column]).mean()
            self.random_same_pk_fk(input_table, 0, onehot=onehot)
        elif strategy == 'correlation':
            self.correlation(self.input_table, self.important_column, 0)
        
        # Sort result by recursion level and index
        self.result.sort(key=lambda x: (x[0], x[1]))

        # Print results and save every table to a file
        print('Level, Index, Primary Key, Columns')
        for (el, col, table) in self.result:
            print(el, " ", col, " ", table.index.names, " ", table.columns, "\n")
            table.to_csv(path + '/table' + str(el) + str(col) + '.csv')

        # Initialise set of tuples in the form (table name, PK column)
        all_tables = []

        # Iterate over tables and fill data
        for (el, col, table) in self.result:
            # Add tables to set
            all_tables.append((str('table' + str(el) + str(col)), table.index.names))

        # Save connections to file
        np.savetxt(path + "/connections.csv", self.connections, delimiter=',', fmt='%s')
        
        # Save tables names with their PK to file
        all_tables = json.dumps(all_tables)
        with open(path + '/tables.json', 'w') as outfile:
            json.dump(all_tables, outfile)

In [6]:
# Initialise divider
dv = Divider(train_data, y_column)

In [7]:
# Apply division
# dv.divide(strategy = "correlation", path = 'output')
# dv.divide(strategy = "random", path = 'output')
dv.divide(strategy = "random", path = 'output', onehot=True)

Level, Index, Primary Key, Columns
0   0   ['PassengerId', 'Survived']   Index(['FK12', 'FK16', 'FK117'], dtype='object') 

1   2   ['PK12']   Index(['FK24', 'FK25'], dtype='object') 

1   6   ['PK16']   Index(['FK28', 'FK215', 'FK216'], dtype='object') 

1   17   ['PK117']   Index(['SibSp'], dtype='object') 

2   4   ['PK24']   Index(['Pclass', 0, 1, 2], dtype='object') 

2   5   ['PK25']   Index(['Age'], dtype='object') 

2   8   ['PK28']   Index(['FK310', 'FK314'], dtype='object') 

2   15   ['PK215']   Index(['Fare'], dtype='object') 

2   16   ['PK216']   Index(['Embarked', 0, 1, 2], dtype='object') 

3   10   ['PK310']   Index(['FK412', 'FK413'], dtype='object') 

3   14   ['PK314']   Index(['Title', 0, 1, 2, 3, 4], dtype='object') 

4   12   ['PK412']   Index(['Sex', 0, 1], dtype='object') 

4   13   ['PK413']   Index(['Parch'], dtype='object') 



In [8]:
from numpy import genfromtxt

# Read data from tables - returns array of [table name, Primary key]
read_tables = []
with open('output/tables.json') as json_file:
    read_tables = json.loads(json.load(json_file))
print(read_tables)

[['table00', ['PassengerId', 'Survived']], ['table12', ['PK12']], ['table16', ['PK16']], ['table117', ['PK117']], ['table24', ['PK24']], ['table25', ['PK25']], ['table28', ['PK28']], ['table215', ['PK215']], ['table216', ['PK216']], ['table310', ['PK310']], ['table314', ['PK314']], ['table412', ['PK412']], ['table413', ['PK413']]]


In [9]:
# Read data from tables - read tables as pandas and set index as Primary key column
read_tables_content = dict()
for [table, pk] in read_tables:
    read_tables_content[table] = pd.read_csv('output/' + table + '.csv', index_col=pk)
    
print(read_tables_content)

{'table00':                       FK12  FK16  FK117
PassengerId Survived                   
1           0          389   696    355
2           1          247   454    423
3           1          539   779    236
4           1          660   483    351
5           0          526   209    643
...                    ...   ...    ...
887         0          322   160    696
888         1          822   440    136
889         0          152   445    809
890         1          811   448    699
891         0          556   260    277

[891 rows x 3 columns], 'table12':       FK24  FK25
PK12            
0       86   785
1      654     7
2      212   413
3      201    47
4      625   203
...    ...   ...
886    265     5
887    137   525
888    858   477
889    129   460
890    392   208

[891 rows x 2 columns], 'table16':       FK28  FK215  FK216
PK16                    
0      883    743    370
1      370    868    564
2       32    627    775
3      151    469    332
4      499    380    348


In [10]:
# Test table reading
table01 = pd.read_csv('output/table00.csv', index_col='PassengerId')
table01

,Survived,FK12,FK16,FK117
PassengerId,,,,
1,0,389,696,355
2,1,247,454,423
3,1,539,779,236
4,1,660,483,351
5,0,526,209,643
...,...,...,...,...
887,0,322,160,696
888,1,822,440,136
889,0,152,445,809


In [11]:
# Read all ((table1, PK), (table2, FK)) relations and add them to a list
read_connections = map(lambda x: ((x[0], x[1]), (x[2], x[3])), genfromtxt('output/connections.csv', delimiter=',', dtype='str'))
read_connections = list(read_connections)
print(read_connections)

[(('table00', 'FK12'), ('table12', 'PK12')), (('table12', 'FK24'), ('table24', 'PK24')), (('table12', 'FK25'), ('table25', 'PK25')), (('table00', 'FK16'), ('table16', 'PK16')), (('table16', 'FK28'), ('table28', 'PK28')), (('table28', 'FK310'), ('table310', 'PK310')), (('table310', 'FK412'), ('table412', 'PK412')), (('table310', 'FK413'), ('table413', 'PK413')), (('table28', 'FK314'), ('table314', 'PK314')), (('table16', 'FK215'), ('table215', 'PK215')), (('table16', 'FK216'), ('table216', 'PK216')), (('table00', 'FK117'), ('table117', 'PK117'))]


In [12]:
# Example of joining tables by going over the relations list
for ((table1, index1), (table2, index2)) in read_connections:
    print(read_tables_content[table1].merge(read_tables_content[table2], left_on=index1, right_on=index2))

     FK12  FK16  FK117  FK24  FK25
0     389   696    355   242   827
1     247   454    423   574   869
2     539   779    236   764   813
3     660   483    351   732   800
4     526   209    643   279   307
..    ...   ...    ...   ...   ...
886   322   160    696   593    82
887   822   440    136   695   852
888   152   445    809   431   209
889   811   448    699   224   715
890   556   260    277    44   149

[891 rows x 5 columns]
     FK24  FK25  Pclass    0    1    2
0      86   785     2.0  0.0  0.0  1.0
1     654     7     1.0  0.0  1.0  0.0
2     212   413     1.0  0.0  1.0  0.0
3     201    47     0.0  1.0  0.0  0.0
4     625   203     0.0  1.0  0.0  0.0
..    ...   ...     ...  ...  ...  ...
886   265     5     2.0  0.0  0.0  1.0
887   137   525     0.0  1.0  0.0  0.0
888   858   477     2.0  0.0  0.0  1.0
889   129   460     2.0  0.0  0.0  1.0
890   392   208     0.0  1.0  0.0  0.0

[891 rows x 6 columns]
     FK24  FK25   Age
0      86   785  19.0
1     654     7  28.